## 생물정보학 및 실습 I final project (sojeong Lee)

#### LIN28 CLIP-seq data를 활용하여 논문 figure와 유사한 형태로 재현하기

* Misson3을 참고하여, 다른 cell의 CLIP-seq data set에서 Fig S2A와 유사한 형태로 재현하는 것을 목표로 함. 
* 원본 데이터에 맞춰서 데이터를 처리하고 시각화해보기

2024-05-30
Human K562 cell Lin28B CLIP-seq data from ENCODE


1. change working directory

In [1]:
%cd /qbio/lsj975/study/down_data/

/qbio/lsj975/study/down_data


In [3]:
# The following commands are run in the terminal

# samtools sort -b -o bam/grch_lin28b_clip.bam grch_lin28b_clip.sorted.bam
# samtools index bam/grch_lin28b_clip.sorted.bam grch_lin28b_clip.sorted.bam.bai


total 994852
drwxr-xr-x 1 lsj975 qbio       146 May 30 19:38 .
drwxr-xr-x 1 lsj975 qbio       288 May 30 19:43 ..
drwxrwxrwx 1 root   qbio       300 May 30 19:47 @eaDir
-rw-r--r-- 1 lsj975 qbio       195 May 30 19:07 files.txt
-rw-r--r-- 1 lsj975 qbio 242463028 May 30 19:10 grch_lin28b_clip.bam
-rw-r--r-- 1 lsj975 qbio    781912 May 30 17:57 GSE37111_clip.txt.gz
-rwxrwxrwx 1 lsj975 qbio 775474314 May 30 19:25 SRR458759.fasta.gz


2. human annotation file에서 miR-let7f1 찾아보기

In [2]:
!zcat /vault/db/gencode/human/latest_release/gencode.v45.annotation.gtf.gz |grep MIRLET7F1

chr9	ENSEMBL	gene	94176347	94176433	.	+	.	gene_id "ENSG00000199072.3"; gene_type "miRNA"; gene_name "MIRLET7F1"; level 3; hgnc_id "HGNC:31483";
chr9	ENSEMBL	transcript	94176347	94176433	.	+	.	gene_id "ENSG00000199072.3"; transcript_id "ENST00000362202.3"; gene_type "miRNA"; gene_name "MIRLET7F1"; transcript_type "miRNA"; transcript_name "MIRLET7F1-201"; level 3; transcript_support_level "NA"; hgnc_id "HGNC:31483"; tag "basic"; tag "Ensembl_canonical";
chr9	ENSEMBL	exon	94176347	94176433	.	+	.	gene_id "ENSG00000199072.3"; transcript_id "ENST00000362202.3"; gene_type "miRNA"; gene_name "MIRLET7F1"; transcript_type "miRNA"; transcript_name "MIRLET7F1-201"; exon_number 1; exon_id "ENSE00001436965.3"; level 3; transcript_support_level "NA"; hgnc_id "HGNC:31483"; tag "basic"; tag "Ensembl_canonical";


** index file을 새로 만들어주고 해당 좌표 라인을 parsing하기

In [5]:
!samtools view -X bam/grch_lin28b_clip.sorted.bam bam/grch_lin28b_clip.sorted.bam.bai chr9:94176347-94176433 -o lin28b_mirlet7f1_clip.bam

In [6]:
!samtools mpileup bam/lin28b_mirlet7f1_clip.bam > pileup/lin28b_mirlet7f1_clip.pileup
!wc -l lin28b_mirlet7f1_clip.pileup

[mpileup] 1 samples in 1 input files
34138 lin28b_mirlet7f1_clip.pileup


In [9]:
!awk '$2 >= 94176347 && $2 <= 94176433 { print $0; }' pileup/lin28b_mirlet7f1_clip.pileup > pileup/lin28b_mirlet7f1_clip-gene.pileup
!tail pileup/lin28b_mirlet7f1_clip-gene.pileup

chr9	94176424	N	453	<C$CCCCCCCA$CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC$CCCCCCCCCC$CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCcccccccccccccccccCCCCCCCCCCCCCCCCCCCCCCCCCCCCCcccccccccccCCCCccccccccccccccccccccccccccccccccctcccCCCCCcccccccccccccCCCCCCCcccccccccC$CCCCCCCCcCCCCCCCCCCcccccccccccccccccccCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCccccccccccccccccccccccccccccccCCCCCcccccccccccccccccCCcccCCCCccccCCcccccccCCCccccccccccccCCcccccccccccccccccCcccccccCCcccCcccCccccccCCCccCcccCCCCccCCc^~c	FkkFkkkkkkk/kkFkFkFFkkkkkkkkkFkkkkgkkaaTak]kkkkkkTWkkPkkkkakkXkFkFkFkFkTJkkkkPgkTkkkkPgkF/kkTkkkBkk/PFgkkkkkTkgFFWkgTBTTFkFakFgkkkkFTkakTkgkkkkkJFkTFkFkkkkFkkkkFFkWFFBaFTFFkFTk>FkFkgkkkkBkBkkFkkPkkkgkkkFkT<gaakkkkFkkkFkkkgkkkkkgkkkkggkkkkkkgkkakFkkkTkkkgaakkkkgkkkkgkkkakFkgkTkTkkgkaFgakkkkkgkkakkTakTkkkkkkkFkkTkkkkkkkkkaTk=kkgkkakkFkTgkkFkF<kkakgkakkTFTgaakkkkFTkkkkakkkkkcXFkBkFkkBT]kBka=kkkkTkkgakak7kkFFkk=kkFkkgkTa>kkkaakkkkkgkkkkkaJkgakT/kWakgk=k
chr9	94176425	N	454	<CCCCCCCCCCCCCCCCCCC$CCCCCCCCCCCCCCCCCCCCCCCC

#### Calculate Shannon entropy

In [10]:
import pandas as pd

pileup = pd.read_csv('pileup/lin28b_mirlet7f1_clip-gene.pileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'])
pileup

,chrom,pos,_ref,count,basereads,quals
0,chr9,94176347,N,13,<TTTTTTTTTTTt,FFgF<FFFFFF/P
1,chr9,94176348,N,13,<CCCCCCCCCCCc,FFaFFFFFFFF/k
2,chr9,94176349,N,13,<AAAAAAAAAAAa,FFaFBFFFFFFFa
3,chr9,94176350,N,13,<G$GGGGGGGGGGg,FFkFFFFFFFFFJ
4,chr9,94176351,N,12,<A$AAAAAAAAAa,FJFFFFFFFFFF
...,...,...,...,...,...,...
82,chr9,94176429,N,305,<A$CCCCCC$CC$CCCCA$CCCC$C$CCCCCCCCCC$CCCA$C$CC...,FkgkkkkaTkkkkak]kkTkkkPakkckgkBFFkkkJJTkFkkkkk...
83,chr9,94176430,N,229,<CCCCACCACTCCCCTA$T$CCCCCCCCCCTCCCCCCCCA$CACA$...,F=kaak]akaaTkgkkPTkkTkkFFFgJggFkgTFFakakFkkkFk...
84,chr9,94176431,N,203,<TTTTTTTTTTTTTTATTTTTTTTTTTTTTTTTTTTTTTTTA$TTT...,FgkTkkTakkggggkkkkJkk<FFkkc=Fkkg/FgakFkkFkkgkk...
85,chr9,94176432,N,200,<GGG$GTGGTGT$GGGGTGGGGGGGGGGT$GGGGGG$GGTGGGGGG...,Fkkkkkaak]kkakkkkkTkk/FFgJJTkkkkakak<kkFkTakkk...


In [11]:
import re
toremove = re.compile('[<>$*#^]')
pileup['matches'] = pileup['basereads'].apply(lambda x: toremove.sub('', x).upper())


In [12]:
pileup[['chrom', 'pos', 'matches']]

,chrom,pos,matches
0,chr9,94176347,TTTTTTTTTTTT
1,chr9,94176348,CCCCCCCCCCCC
2,chr9,94176349,AAAAAAAAAAAA
3,chr9,94176350,GGGGGGGGGGGG
4,chr9,94176351,AAAAAAAAAAA
...,...,...,...
82,chr9,94176429,ACCCCCCCCCCCCACCCCCCCCCCCCCCCCCCACCCCCCCCCCCCC...
83,chr9,94176430,CCCCACCACTCCCCTATCCCCCCCCCCTCCCCCCCCACACACCCAC...
84,chr9,94176431,TTTTTTTTTTTTTTATTTTTTTTTTTTTTTTTTTTTTTTTATTTTT...
85,chr9,94176432,GGGGTGGTGTGGGGTGGGGGGGGGGTGGGGGGGGTGGGGGGGGGGG...


In [13]:
import numpy as np

total = pileup['matches'].apply(lambda n: len(n) )
base_count = pileup['matches'].apply(lambda x: [x.count(base) for base in ['A', 'T', 'G', 'C']])
prob = [-(base_count[i]/total[i])*np.log2(base_count[i]/total[i]) if total[i] != 0 else [0,0,0,0] for i in range(len(base_count))]
pileup['score'] = pd.DataFrame(prob).fillna(0).sum(axis=1)
pileup['end'] = pileup['pos']+1

pileup[['chrom', 'pos', 'end', 'score']]

/tmp/ipykernel_2135880/620243297.py:7: RuntimeWarning: divide by zero encountered in log2
  prob = [-(base_count[i]/total[i])*np.log2(base_count[i]/total[i]) if total[i] != 0 else [0,0,0,0] for i in range(len(base_count))]
/tmp/ipykernel_2135880/620243297.py:7: RuntimeWarning: invalid value encountered in multiply
  prob = [-(base_count[i]/total[i])*np.log2(base_count[i]/total[i]) if total[i] != 0 else [0,0,0,0] for i in range(len(base_count))]


,chrom,pos,end,score
0,chr9,94176347,94176348,0.000000
1,chr9,94176348,94176349,0.000000
2,chr9,94176349,94176350,0.000000
3,chr9,94176350,94176351,0.000000
4,chr9,94176351,94176352,0.000000
...,...,...,...,...
82,chr9,94176429,94176430,0.382821
83,chr9,94176430,94176431,1.093729
84,chr9,94176431,94176432,0.262001
85,chr9,94176432,94176433,0.689403


In [14]:
bedf = pileup[['chrom', 'pos', 'end', 'score']]
bedf.describe()

,pos,end,score
count,8.700000e+01,8.700000e+01,87.000000
mean,9.417639e+07,9.417639e+07,0.110228
std,2.525866e+01,2.525866e+01,0.182416
min,9.417635e+07,9.417635e+07,0.000000
25%,9.417637e+07,9.417637e+07,0.000000
50%,9.417639e+07,9.417639e+07,0.040149
75%,9.417641e+07,9.417641e+07,0.124692
max,9.417643e+07,9.417643e+07,1.093729


In [15]:
bedf.to_csv('/qbio/lsj975/study/down_data/UCSC/mirlet7f1_shannon_entropy.bedGraph', sep = '\t', index=False, header=False)

When upload wig file to UCSC genome brower, wrtie track description including track type=bedGraph, and other annotation like name, description. 